In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 1_parse_sublink_trees.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstring:
'''Parse sublink trees of MW analogs and find progenitors of high mass ratio 
mergers. Track those systems back in time and records their particles.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle, glob

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import plot as pplot
from tng_dfs import tree as ptree
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/2_merger_sample/1_parse_sublink_trees/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
# tree_primary_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_primaries.pkl')
# with open(tree_primary_filename,'rb') as handle: 
#     tree_primaries = pickle.load(handle)
# tree_major_mergers_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_major_mergers.pkl')
# with open(tree_major_mergers_filename,'rb') as handle:
#     tree_major_mergers = pickle.load(handle)
n_mw = len(mwsubs)

### Get the sublink trees

In [ ]:
tree_dir = data_dir+'mw_analogs/sublink_trees/full/'
sublink_files = glob.glob(tree_dir+'*.hdf5')
sublink_files_mask = mwsubs_vars['bulge_disk_fraction_mask']
sublink_files = [sublink_files[indx] for indx in np.where(sublink_files_mask)[0]]

### Create TreeMajorMerger and TreePrimary objects and save
The preferred method is the mass ratio at the maximum stellar mass, interpolating when the mass of the primary decreases. 

In [ ]:
# Keywords and scheme
mass_ratio_threshold = 1./20.
check_descends_to_main = True
scheme = 'mratio_at_tmax'
particle_type = 'stars'
mass_star_dict = {'key':'SubhaloMassType','ptype':particle_type}
scheme_kwargs = {'mass_tmax_key':mass_star_dict, 
                 'mass_mratio_key':mass_star_dict,
                 'mass_ratio_threshold':mass_ratio_threshold,
                 'mask_main_branch_mass_growing':False,
                 'use_interpolated_main_branch_mass':True}

# Figure pathing, this should probably reference mask_main_branch_mass_growing 
# and use_interpolated_main_branch_mass keywords from above
scheme_dir = 'mratio_at_tmax_interp_mass'
os.makedirs(os.path.join(fig_dir, scheme_dir, particle_type), 
    exist_ok=True)

tree_primaries = []
tree_major_mergers = []

for i in range(n_mw):
    print('Working on MW analog '+str(i+1)+' of '+str(n_mw), end='\r')

    # Get the tree and find major mergers
    tree = ptree.SublinkTree(sublink_files[i])
    z0_sid = tree.get_property('SubfindID')[0]
    tree.find_where_main_branch_mass_growing(_check=True)
    tree.find_mapping_secondary_to_main_branch()
    major_merger_mlpid,major_merger_mass_ratio,major_merger_mass_ratio_snap = \
        tree.find_major_mergers(scheme=scheme,
            scheme_kwargs=scheme_kwargs,
            check_descends_to_main=check_descends_to_main)
    n_major = len(major_merger_mlpid)

    major_merger_snapnum = []
    major_merger_dm_mass_ratio = []
    dm_interp = tree.get_main_branch_mass_growing_interpolator(mass_key='Mass')
    # Loop over the secondaries and calculate DM mass ratios + merger snapnums
    for j in range(n_major):
        
        # Get the merger snapshot number 
        major_merger_mlpid_mask = \
            tree.get_property('MainLeafProgenitorID') == major_merger_mlpid[j]
        major_merger_snapnum.append(np.max(tree.get_property('SnapNum')
            [major_merger_mlpid_mask]))

        # Get the DM mass ratio at the same snapshot as the merger was detected.
        secondary_snapnum_mask = (tree.get_property('SnapNum')
            [major_merger_mlpid_mask] == major_merger_mass_ratio_snap[j])
        secondary_dm_mass = (tree.get_property('SubhaloMassType')[:,1]
            [major_merger_mlpid_mask][secondary_snapnum_mask])
        primary_dm_mass = dm_interp(major_merger_mass_ratio_snap[j])
        major_merger_dm_mass_ratio.append(float(secondary_dm_mass/primary_dm_mass))

    # Create plots
    main_leaf_progenitor_id = tree.get_property('MainLeafProgenitorID')
    subfind_id = tree.get_property('SubfindID')
    mass = tree.get_property('Mass')
    mass_star = tree.get_property('SubhaloMassType', ptype='stars')
    snap = tree.get_property('SnapNum')

    this_fig_dir = os.path.join(fig_dir, scheme_dir, particle_type,
        str(z0_sid))
    os.makedirs(this_fig_dir,exist_ok=True)

    for j in range(len(major_merger_mlpid)):
        fig,axs = pplot.plot_merger_information(tree, 
            mlpid=major_merger_mlpid[j],
            snap_mass_ratio=major_merger_mass_ratio_snap[j],
            main_leaf_progenitor_id=main_leaf_progenitor_id,
            subfind_id=subfind_id, mass=mass, mass_star=mass_star, snap=snap)
        this_fig_name = os.path.join(this_fig_dir,
            str(major_merger_mlpid[j])+'_merger_information.png')
        fig.savefig(this_fig_name, dpi=300, bbox_inches='tight')
        if not show_plots: plt.close(fig)
    
    fig,axs = pplot.plot_all_merger_traces(tree, major_merger_mlpid)
    this_fig_name = os.path.join(this_fig_dir, 'all_merger_traces.png')
    fig.savefig(this_fig_name, dpi=300, bbox_inches='tight')
    if not show_plots: plt.close(fig)

    # Loop over the secondaries and create TreeMajorMerger objects
    _tree_major_mergers = []
    for j in range(n_major):

        tree_major_merger_kwargs = {
            'secondary_mlpid':major_merger_mlpid[j],
            'primary_mlpid':tree.main_branch_mlpid,
            'star_mass_ratio':major_merger_mass_ratio[j],
            'star_mass_ratio_snapnum':major_merger_mass_ratio_snap[j],
            'dm_mass_ratio':major_merger_dm_mass_ratio[j],
            'dm_mass_ratio_snapnum':major_merger_mass_ratio_snap[j],
            'merger_snapnum':major_merger_snapnum[j],
            'scheme':scheme,
            'scheme_kwargs':scheme_kwargs,
            'tree_filename':sublink_files[i],
        }
        _tree_major_mergers.append(
            ptree.TreeMajorMerger(**tree_major_merger_kwargs)
        )
    tree_major_mergers.append(_tree_major_mergers)

    # Make the primary
    tree_primary_kwargs = {
        'primary_mlpid':tree.main_branch_mlpid,
        'tree_filename':sublink_files[i]
    }
    tree_primaries.append(
        ptree.TreePrimary(tree_major_mergers=_tree_major_mergers,
                          **tree_primary_kwargs)
    )

# Save
major_merger_dir = os.path.join(mw_analog_dir,'major_mergers/')
os.makedirs(major_merger_dir,exist_ok=True)
with open(os.path.join(major_merger_dir,'tree_primaries.pkl'),'wb') as f:
    pickle.dump(tree_primaries, f)
with open(os.path.join(major_merger_dir,'tree_major_mergers.pkl'),'wb') as f:
    pickle.dump(tree_major_mergers, f)